In [9]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0 pyspark-shell'

from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, element_at, when

In [3]:
TOPIC_NAME = "topic_1"
HOST_IP = "192.168.20.6"

In [4]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('[Demo] Spark Streaming from Kafka into MongoDB')
    .getOrCreate()
)

:: loading settings :: url = jar:file:/Users/petermok/opt/miniconda3/envs/fit3182/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/petermok/.ivy2/cache
The jars for the packages stored in: /Users/petermok/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-89008c3a-c48d-44c3-b4c2-f0c5052761d3;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305

In [10]:
p1_topic_name = "producer_1"
p1_stream_df = (
    spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', f'{HOST_IP}:9092')
    .option('subscribe', p1_topic_name)
    .load()
)
p1_stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [14]:
p1_output_stream_df = (
    p1_stream_df
    .select(                                      # 1
        split(
            p1_stream_df.value.cast('string'),
            ':'
        )
        .alias('data')
    )
    .withColumn('data', element_at('data', 2))    # 2
    .withColumn(
        'data',
        (
            when( col('data') == '', '*' )        # 3A
            .otherwise( col('data') )             # 3B
        )
    )
)
p1_output_stream_df.printSchema()

root
 |-- data: string (nullable = true)



In [11]:
class DbWriter:

    def __init__(self):
        self.db = None
        self.mongo_client = None

    # called at the start of processing each partition in each output micro-batch
    def open(self, partition_id, epoch_id):
        self.mongo_client = MongoClient(
            host=f'{"localhost"}',
            port=27017
        )
        self.db = self.mongo_client.fit3182_assignment_db
        return True

    # called once per row of the result dataframe
    def process(self, row):
        self.db[TOPIC_NAME].insert_one(row.asDict())

    # called once all rows have been processed (possibly with error)
    def close(self, err):
        self.mongo_client.close()


In [17]:
db_writer = (
    p1_output_stream_df
    .writeStream
    .outputMode('append')
    .foreach(DbWriter())
)

console_logger = (
    p1_output_stream_df
    .writeStream
    .outputMode('append')
    .format('console')
)

In [18]:
writer = console_logger

In [20]:
from time import sleep
from pyspark.errors import StreamingQueryException

try:
    query = writer.start()
    query.awaitTermination()
    sleep(10)
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopped query')
except StreamingQueryException as exc:
    print(exc)
finally:
    query.stop()


23/06/02 02:18:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/nk/pq_9ypcs6_x5jdx99mrtszc80000gn/T/temporary-5d31f9ba-8d54-407a-b1e7-409464891844. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/06/02 02:18:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/06/02 02:18:08 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/06/02 02:18:08 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/06/02 02:18:08 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/06/02 02:18:08 WARN AdminClientConfig: The configuration '

-------------------------------------------
Batch: 0
-------------------------------------------
+----+
|data|
+----+
+----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----------+
|       data|
+-----------+
| 159, dtype|
+-----------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----------+
|       data|
+-----------+
| 283, dtype|
+-----------+



-------------------------------------------
Batch: 3
-------------------------------------------
+----------+
|      data|
+----------+
| 55, dtype|
+----------+



-------------------------------------------
Batch: 4
-------------------------------------------
+-----------+
|       data|
+-----------+
| 197, dtype|
+-----------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/petermok/opt/miniconda3/envs/fit3182/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/petermok/opt/miniconda3/envs/fit3182/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/petermok/opt/miniconda3/envs/fit3182/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Interrupted by CTRL-C. Stopped query
